In [1]:
import os
import shutil
import warnings
import urllib.request as request
from contextlib import closing

import cupy

import dask_cudf
import dask_ml

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

from nvidia.cheminformatics.chembldata import ChEmblData

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

# Download ChEMBL database

In [2]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [3]:
enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

n_clusters = 3
n_neighbors=100
pca_comps = 64

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]

# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
client

Client Scheduler: ucx://127.0.0.1:36723 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 1 Cores: 1 Memory: 33.27 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [4]:
%%time
chem_data = ChEmblData(db_file=db_file)

# Fetch and generate Finger prints
ddf = chem_data.fetch_all_props(num_recs=100000)

CPU times: user 103 ms, sys: 7.85 ms, total: 111 ms
Wall time: 103 ms


# Clustering

In [5]:
%%time
dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()

pca = cuDaskPCA(client=client, n_components=pca_comps)
df_fingerprints = pca.fit_transform(dcudf)

kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
kmeans_float.fit(df_fingerprints)
kmeans_labels = kmeans_float.predict(df_fingerprints)

CPU times: user 1.94 s, sys: 495 ms, total: 2.43 s
Wall time: 49.9 s


In [6]:
%%time
local_model = cuUMAP()
X_train = df_fingerprints.compute()
local_model.fit(X_train)

umap_model = cuDaskUMAP(local_model,
                        n_neighbors=n_neighbors,
                        a=1.0,
                        b=1.0,
                        learning_rate=1.0,
                        client=client)
Xt = umap_model.transform(df_fingerprints)

dcudf['x'] = Xt[0]
dcudf['y'] = Xt[1]
dcudf['cluster'] = kmeans_labels

dcudf.head(5)

CPU times: user 3.23 s, sys: 267 ms, total: 3.5 s
Wall time: 4.99 s


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,x,y,cluster
molregno,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.027734,-1.563151,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.195816,-1.660381,0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.743623,-1.487076,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.175279,-1.648877,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.061382,-1.441463,0


In [7]:
%%time

def show_cluster_plot(ldcudf, title='UMAP'):
    umap_fig = figure(title=title, width=800)
    clusters = ldcudf['cluster'].unique().compute().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)
        cdf = cdf.compute()

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)

show_cluster_plot(dcudf)

CPU times: user 2.79 s, sys: 355 ms, total: 3.15 s
Wall time: 16.6 s


# Re-clustering

In [ ]:
%%time
cluster = 1 # <- PLEASE CHANGE CLUSTER AS REQUIRED
query = 'cluster == ' + str(cluster)

print(type(tcdf))
print('Reclustering...')
tcdf = tcdf.drop(['x', 'y', 'cluster'], axis=1)

print('Computing KMEANS...')
pca = cuDaskPCA(client=client, n_components=pca_comps)
df_fingerprints = pca.fit_transform(tcdf)

kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
kmeans_float.fit(df_fingerprints)
kmeans_labels = kmeans_float.predict(df_fingerprints)

print('Computing UMAP...')
local_model = cuUMAP()
X_train = tcdf.compute()
local_model.fit(X_train)

umap_model = cuDaskUMAP(local_model,
                        n_neighbors=n_neighbors,
                        a=1.0,
                        b=1.0,
                        learning_rate=1.0,
                        client=client)
Xt = umap_model.transform(tcdf)

# Add back the column required for plotting and to correlating data
# between re-clustering
tcdf['x'] = Xt[0]
tcdf['y'] = Xt[1]
tcdf['cluster'] = kmeans_labels
tcdf['id'] = tcdf.index

print('Plotting...')
show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))